# World Cup schedule

#### Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

In [5]:
results_dicts = pd.read_json(
    "https://api.fifa.com/api/v1/calendar/matches?idSeason=255711&count=500&language=en"
)["Results"]

In [7]:
match_dict = {key: value for key, value in results_dicts.items() if len(value) > 1}

In [8]:
src = pd.DataFrame(match_dict).T

In [9]:
src.columns = src.columns.str.lower()

In [10]:
src[["locale", "group_name"]] = pd.json_normalize(src["groupname"])[0].apply(pd.Series)

In [11]:
src["home_team"] = pd.json_normalize(src["home"])["Abbreviation"]
src["away_team"] = pd.json_normalize(src["away"])["Abbreviation"]

In [12]:
src[["locale", "stage_name"]] = pd.json_normalize(src["stagename"])[0].apply(pd.Series)

In [13]:
src_slim = src[
    ["idmatch", "date", "matchnumber", "group_name", "home_team", "away_team"]
].copy()

In [14]:
src_slim["local_date"] = pd.to_datetime(src_slim["date"]).dt.strftime("%Y-%m-%d")
src_slim["local_time"] = pd.to_datetime(src_slim["date"]).dt.strftime("%H:%M:%S")

In [15]:
src_slim["pst_date"] = (
    pd.to_datetime(src_slim["date"]).dt.tz_convert("US/Pacific").dt.strftime("%Y-%m-%d")
)

In [16]:
src_slim["pst_time"] = (
    pd.to_datetime(src_slim["date"]).dt.tz_convert("US/Pacific").dt.strftime("%H:%M:%S")
)

In [17]:
src_slim.drop("date", axis=1, inplace=True)

In [19]:
df = src_slim.copy()

----

In [20]:
df.head(10)

,idmatch,matchnumber,group_name,home_team,away_team,local_date,local_time,pst_date,pst_time
0,400128082,1,Group A,QAT,ECU,2022-11-21,16:00:00,2022-11-21,08:00:00
1,400235449,2,Group A,SEN,NED,2022-11-21,10:00:00,2022-11-21,02:00:00
2,400235458,3,Group B,ENG,IRN,2022-11-21,13:00:00,2022-11-21,05:00:00
3,400235455,4,Group B,USA,NaN,2022-11-21,19:00:00,2022-11-21,11:00:00
4,400235470,5,Group D,FRA,NaN,2022-11-22,19:00:00,2022-11-22,11:00:00
5,400235466,6,Group D,DEN,TUN,2022-11-22,13:00:00,2022-11-22,05:00:00
6,400235463,7,Group C,MEX,POL,2022-11-22,16:00:00,2022-11-22,08:00:00
7,400235461,8,Group C,ARG,KSA,2022-11-22,10:00:00,2022-11-22,02:00:00
8,400235477,9,Group F,BEL,CAN,2022-11-23,19:00:00,2022-11-23,11:00:00
9,400235472,10,Group E,ESP,NaN,2022-11-23,16:00:00,2022-11-23,08:00:00


In [34]:
dates = pd.DataFrame(df.local_date.astype(str).unique(), columns=["match_date"])

In [37]:
dates["match_date"].to_list()

['2022-11-21',
 '2022-11-22',
 '2022-11-23',
 '2022-11-24',
 '2022-11-25',
 '2022-11-26',
 '2022-11-27',
 '2022-11-28',
 '2022-11-29',
 '2022-11-30',
 '2022-12-01',
 '2022-12-02',
 '2022-12-03',
 '2022-12-04',
 '2022-12-05',
 '2022-12-06',
 '2022-12-09',
 '2022-12-10',
 '2022-12-13',
 '2022-12-14',
 '2022-12-17',
 '2022-12-18']

---

## Export

In [38]:
df.to_csv("data/processed/world_cup_matches_schedule.csv", index=False)